In [ ]:
import numpy as np
import pandas as pd
import os
import openpyxl
import tensorflow as tf
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
from pandas import concat
from tensorflow.keras.layers import LSTM,Dense,GRU,Dropout,SimpleRNN
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense


In [ ]:
'''
from google.colab import drive
drive.mount('/content/drive')
'''

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
import random as rn
seed_num =42
np.random.seed(seed_num)
rn.seed(seed_num)
tf.random.set_seed(seed_num)

In [ ]:
n_hours = 3
n_features = 43
'''
train_num_24 = 2789
train_num_25 = 2892
train_num_19 = 1934
'''
train_num_24 = 2788
train_num_25 = 2891
train_num_19 = 1933
train_num_26= 3052

In [ ]:
from google.colab import drive
filename = '/content/rain_data.xlsx'

df = pd.read_excel(filename)

In [ ]:
# df_metrix 만드는 과정
df_matrix = df.to_numpy()


df_matrix = df_matrix.astype('float32')
df_matrix = df_matrix[:,5:]

scaler = MinMaxScaler(feature_range=(0, 1))
df_matrix = scaler.fit_transform(df_matrix)
print(df_matrix)


# print(df_matrix.shape)
df_matrix1 = df_matrix[0:train_num_25, :] # 주 numpy
# df_matrix2 = df_matrix[train_num_24-4:train_num_25,5:] 
y = df_matrix[n_hours:train_num_25,42:43]
ans = df_matrix[train_num_24:train_num_25,5:6]

print(y.shape)
print(y)
print(df_matrix[0,0])

In [ ]:
# print(df_matrix)
lag = series_to_supervised(df_matrix1, n_hours,1)

# print(lag)
print(lag.shape[1])
# for i in range(215,258):
#     lag.drop(lag.columns[[i]], axis = 1, inplace = True)


lag = lag.values

tmp = lag.shape[1]

lag = lag[:,0:tmp-1]
print(lag.shape)
lag


In [ ]:
# n_train_hours = train_num_19
train_X = lag
# test_X = lag[n_train_hours:, :]
# train_y = y[:n_train_hours, :]
# test_y = y[n_train_hours:, :]

In [ ]:
n_obs = n_hours * (n_features)
# train_X, train_y = train[:, :n_obs], train[:, -130]
print('train_X: ',train_X)
# print('train_y: ',train_y)
# test_X, test_y = test[:, :n_obs], test[:, -130]
#print('test_y: ',test_y)
# print(train_X.shape, len(train_X), test_X.shape)
garo = train_X.shape[1]
print(garo)
# reshape input to be 3D [samples, timesteps, features]
print(train_X.shape,y.shape)
train_X = train_X.reshape((train_X.shape[0],1, train_X.shape[1]))
# test_X = test_X.reshape((test_X.shape[0],1, test_X.shape[1]))

# print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
#print('test_y: ',test_y)

In [ ]:
# print(df_matrix[train_num_25-1][42])
plt.figure(figsize=(20, 10))
# sns.lineplot(y=df_scaled['유입량'], x=df['일자'])
# plt.xlabel('TIME')
# plt.ylabel('AMOUNT')
plt.plot(df['유입량'])
plt.xlabel('TIME')
plt.ylabel('AMOUNT')

In [ ]:
import os
from keras import optimizers

print(train_num_25,train_num_26)
for i in range(train_num_25,train_num_26):
  print(i)
  model = Sequential()
  model.add(LSTM(20, input_shape=(train_X.shape[1],train_X.shape[2]), activation='tanh', return_sequences=False))
  model.add(Dense(1))

  optimizer = tf.keras.optimizers.Adam(lr=0.0001)

  model.compile(loss='mean_squared_error', optimizer='adam')
#early_stop = EarlyStopping(monitor='val_loss', patience=3)
#model_path = 'model'
#filename = os.path.join(model_path, 'tmp_checkpoint.h5')
#checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

  history = model.fit(train_X, y, 
                                    epochs=50, 
                                    batch_size=32,
                                    # validation_data=(test_X, test_y), 
                                    )

  
  tmp = df_matrix[i-3:i+1,:]
  tmp=tmp.reshape(1,172)
  
  # print(tmp.shape)
  tmp = tmp[:,:-1]
  # print(tmp)
  tmp = tmp.reshape((tmp.shape[0],1,tmp.shape[1]))
  # print(tmp)
  ans = model.predict(tmp)
  df_matrix[i][42]=ans
  # print(ans)
  y = np.concatenate((y,ans),axis =0) #정답 붙이기


  # print(train_X.shape,tmp.shape)
  train_X = np.concatenate((train_X,tmp),axis =0)
  # print(train_X)
  # print(y)



In [ ]:

df_matrix =scaler.inverse_transform(df_matrix)

real = df_matrix[train_num_25:train_num_26,42]
print(real)


In [ ]:
one = df_matrix[train_num_25:train_num_26,0]
two = df_matrix[train_num_25:train_num_26,1]
three = df_matrix[train_num_25:train_num_26,2]
four = df_matrix[train_num_25:train_num_26,3]
five = df_matrix[train_num_25:train_num_26,4]

plt.plot(one)
plt.plot(two)
plt.plot(three)
plt.plot(four)
plt.plot(five)
plt.plot(real)
plt.show()

In [ ]:
from openpyxl import Workbook

write_wb = Workbook()
write_ws = write_wb.create_sheet("결과")

write_ws = write_wb.active
real= real.tolist()
# print(len(real))
for i in range(len(real)):
  write_ws.cell(i+1,1,real[i])
write_wb.save('/content/결과1.xlsx')

In [ ]:


optimizer = tf.keras.optimizers.Adam(lr=0.0001)

model.compile(loss='mean_squared_error', optimizer='adam')
#early_stop = EarlyStopping(monitor='val_loss', patience=3)
#model_path = 'model'
#filename = os.path.join(model_path, 'tmp_checkpoint.h5')
#checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit(train_X, train_y, 
                                    epochs=50, 
                                    batch_size=32,
                                    # validation_data=(test_X, test_y), 
                                    )

In [ ]:
yhat = model.predict(test_X)


inv_yhat = df_matrix
a=0
for i in range(train_num_19+3,train_num_24):
  df_matrix[i][42]=yhat[a]
  a+=1

inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[train_num_19+3:train_num_24,42:43]
print(inv_yhat)


In [ ]:
np.set_printoptions(suppress=True)

print(inv_yhat)
print(inv_yhat.shape)

In [ ]:

inv_y = scaler.inverse_transform(df_matrix)
inv_y = inv_y[train_num_19+3:train_num_24,42:43]


print(inv_y)
print(inv_y.shape)

In [ ]:
print(inv_yhat[0])

In [ ]:
plt.plot(inv_yhat)
plt.plot(inv_y)
plt.show()

In [ ]:
RMSE = mean_squared_error(inv_yhat,inv_y)**0.5
RMSE

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(history.history['loss'],label = 'train')
plt.plot(history.history['val_loss'], label = 'test')

plt.show()

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(test_y, label = 'actual')
plt.plot(pred, label = 'prediction')
plt.legend()
plt.show()